In [1]:
# NOTEBOOK parameters
# see nbrun docs on github

IMG_SHAPE = (150,150)
BATCH_SIZE = 32
EPOCH = 15
MODEL = 'Inception-ResNet V2 Model' # 'Lite Test' # 
DATA_GEN_CONFIG = 'Baseline Augmentation'
OPT = 'opt-adam-lr2e-05'
AUTO=False

In [2]:
!curl -L -o noahs_utils.py https://gist.github.com/bantucaravan/1956003e25c056c550a088542b41dc91/raw/noahs_utility_funcs.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12726  100 12726    0     0  35369      0 --:--:-- --:--:-- --:--:--  443k


In [2]:
import os
import glob
import re
import datetime as dt
from collections import defaultdict
import sys
import importlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

#import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Dropout, Conv2D, MaxPooling2D, Flatten, Dense

from noahs_utils import *
import build_data_gens
import build_models

In [4]:
if False:
    import noahs_utils
    importlib.reload(noahs_utils)

In [5]:
if False:
    import GIST_git_clone.noahs_utility_funcs
    from GIST_git_clone.noahs_utility_funcs import *
    importlib.reload(GIST_git_clone.noahs_utility_funcs)

## Explore Building Finetuning model - InceptionResNetV2

In [6]:
#half_cooked_ts = '2020-02-07_03h46m48s' # InceptionResNetV2 60eps opt-adam
half_cooked_ts = '2020-02-09_01h00m31s' # 35 ep InceptionResNEtV2 w/ Dropout post Base , opt-adam
get_loss = lambda x: float(re.findall(r'_(\d\.\d{2})_', x)[0])
pat = '../saved models/model_epoch*_*_%s.h5' %(half_cooked_ts)
modelpaths = sorted(glob.glob(pat), key=get_loss)
modelpaths

['../saved models/model_epoch01_0.63_2020-02-09_01h00m31s.h5',
 '../saved models/model_epoch06_0.70_2020-02-09_01h00m31s.h5',
 '../saved models/model_epoch03_0.70_2020-02-09_01h00m31s.h5',
 '../saved models/model_epoch08_0.71_2020-02-09_01h00m31s.h5',
 '../saved models/model_epoch18_0.72_2020-02-09_01h00m31s.h5',
 '../saved models/model_epoch30_0.73_2020-02-09_01h00m31s.h5',
 '../saved models/model_epoch31_0.74_2020-02-09_01h00m31s.h5']

In [11]:
bestmodelpath = modelpaths[-1]
print(bestmodelpath)
model = load_model(bestmodelpath)

../saved models/model_epoch31_0.74_2020-02-09_01h00m31s.h5


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 1536)              54336736  
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               393472    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1542      
Total params: 54,731,750
Trainable params: 395,014
Non-trainable params: 54,336,736
_________________________________________________________________


In [17]:
len(model.trainable_weights)

4

In [18]:
model.layers

In [15]:
for i in model.layers:
    print(i.name, 'Trainable:', i.trainable)

inception_resnet_v2 Trainable: True
dropout Trainable: True
dense Trainable: True
dense_1 Trainable: True


In [26]:
sum(i.trainable for i in model.layers[0].layers)

20

In [25]:
for i, l in enumerate(model.layers[0].layers):
    print(i, l.name, 'Trainable:', l.trainable)

0 input_1 Trainable: True
1 conv2d Trainable: False
2 batch_normalization Trainable: False
3 activation Trainable: False
4 conv2d_1 Trainable: False
5 batch_normalization_1 Trainable: False
6 activation_1 Trainable: False
7 conv2d_2 Trainable: False
8 batch_normalization_2 Trainable: False
9 activation_2 Trainable: False
10 max_pooling2d Trainable: False
11 conv2d_3 Trainable: False
12 batch_normalization_3 Trainable: False
13 activation_3 Trainable: False
14 conv2d_4 Trainable: False
15 batch_normalization_4 Trainable: False
16 activation_4 Trainable: False
17 max_pooling2d_1 Trainable: False
18 conv2d_8 Trainable: False
19 batch_normalization_8 Trainable: False
20 activation_8 Trainable: False
21 conv2d_6 Trainable: False
22 conv2d_9 Trainable: False
23 batch_normalization_6 Trainable: False
24 batch_normalization_9 Trainable: False
25 activation_6 Trainable: False
26 activation_9 Trainable: False
27 average_pooling2d Trainable: False
28 conv2d_5 Trainable: False
29 conv2d_7 Trainabl

In [24]:
#Set final Inception-ResNet-C Module to trainable
#see paper: https://arxiv.org/pdf/1602.07261v2.pdf
#Model summary (saved) and model graph (saved)
#after 'block8_9_ac', index 761
for i, l in enumerate(model.layers[0].layers):
    if i > 761:
        l.trainable = True

In [ ]:
sum(i.trainable for i in model.layers[0].layers)

In [29]:
# this is just a place holder so trainable weight count will be updated
model.compile(optimizer='adam', 
            loss='categorical_crossentropy', 
            metrics=['accuracy'])

In [30]:
model.summary()
## I am wondering is actual param count is missing some things...?

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 1536)              54336736  
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               393472    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1542      
Total params: 54,731,750
Trainable params: 5,625,990
Non-trainable params: 49,105,760
_________________________________________________________________


### End: Explore Finetuning